In [1]:
import matplotlib.pyplot as plt
import numpy as np
import egg.core as core
import pandas as pd

from utils.analysis_tools_lazimpa import *
from utils.analysis_from_interaction import *
from archs import Sender, Receiver
from train import loss
from models_lazimpa import LazImpaSenderReceiverRnnGS
from dataset import DataSet
import torch.nn.functional as F

In [54]:
dataset = '(3,4)'
values = 4
attributes = 3
n_epochs = 300
path = 'results/' + dataset + '_game_size_10_vsf_3'
context_unaware = False # whether original or context_unaware simulations are evaluated
lazy = True # whether the lazy agent is used
impatient = True
setting = ""
if not lazy:
    start = 'impatience_' if impatient else ''
    if context_unaware:
        setting = start + 'context_unaware'
    else:
        setting = start + 'context_aware'
        if impatient == False:
            setting = 'standard'
else:
    start = 'lazimpa' if impatient else 'lazy'
    if context_unaware:
        setting = start + '_context_unaware'
    else:
        setting = start + '_context_aware'
setting
run = 3
setting

'lazimpa_context_aware'

In [72]:
# Start what kind of variables do I need for attribute visualization

interaction = load_interaction(path,setting,run,n_epochs)
concepts, context_condition = retrieve_concepts_context(interaction,values)
messages = retrieve_messages(interaction,False,remove_after_eos=True)
message_length = MessageLengthHierarchical.compute_message_length(messages)
unique_messages, frequencies = torch.unique(messages,dim=0,return_counts=True) # frequency of message outside of this usage case
unique_length = MessageLengthHierarchical.compute_message_length(unique_messages)
percent_wrong, bool_wrong = percent_wrong_from_interaction(interaction)
listener,_,model = load_listener(path, setting, run, attributes, values, context_unaware)
sender = model.sender 

m = [f"{ *m, }" for m in messages.numpy()] # as str for hashing
l = message_length.numpy()
cx = context_condition
fint = [list(map(int,list(f))) for c,f in concepts]
fix = [f"{v}" for v in fint] # fix as string
c = [list( map(int, list( [a if fa == 1. else -1 for a,fa in zip(c[0],f) ] ))) for c,f in concepts]
um = [f"{ *m, }" for m in unique_messages.numpy()]
ul = unique_length.numpy()
f = frequencies.numpy()

# make one DataFrame containing all the data we have
in_list = []
for i in range(message_length.shape[0]):
    content = [int(sl) if sl == 1. else "" for sl in list(fint[i])]
    content.extend((fix[i],fint[i], np.where(concepts[i][1]==1.0)[0].tolist(), sum(fint[i]), c[i], cx[i], m[i], messages.numpy()[i], l[i],percent_wrong[i].item(),bool_wrong[i].item()))
    in_list.append(content)
df = pd.DataFrame(in_list,columns=["fixed 1", "fixed 2", "fixed 3","fixed", "fixed int","which fixed indices", "nr attributes", "concept", "context condition", "message", "message int", "message length", "wrong %", "wrong some"])
unique_df = pd.DataFrame([(m,f) for m,f in zip(list(um),list(f))],columns=["message", "usage frequency"])
df = df.merge(unique_df,how='right',on="message")
df

results/(3,4)_game_size_10_vsf_3/lazimpa_context_aware/3/interactions/train/epoch_300/interaction_gpu0
torch.Size([1792, 20, 12])


,fixed 1,fixed 2,fixed 3,fixed,fixed int,which fixed indices,nr attributes,concept,context condition,message,message int,message length,wrong %,wrong some,usage frequency
0,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 2]",2,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,10,1,70
1,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 2]",2,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,30,1,70
2,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[3, 1, 3]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,5,1,70
3,,1,1,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 1, 2]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,20,1,70
4,1,,1,"[1, 0, 1]","[1, 0, 1]","[0, 2]",2,"[2, -1, 2]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,30,1,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",1,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17
1788,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",0,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17
1789,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",1,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17
1790,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",0,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17


In [56]:
# remove all values with wrong some != 0 # in test removes ca 350 from 1784 rows
df_correct = df[df['wrong some'] == 0]

# adjust frequencies
unique_messages, frequencies = torch.unique(torch.Tensor(df_correct['message int'].array),dim=0,return_counts=True) # frequency of message outside of this usage case
f_c = frequencies.numpy()
um_c = [f"{ *m, }" for m in unique_messages.numpy().astype(np.int32)]
ul_c = unique_length.numpy()
unique_df = pd.DataFrame([(m,f_c) for m,f_c in zip(list(um_c),list(f))],columns=["message", "correct frequency"])
df_adjusted = df_correct.merge(unique_df,how='right',on="message")
df = df_adjusted
df

,fixed 1,fixed 2,fixed 3,fixed,fixed int,which fixed indices,nr attributes,concept,context condition,message,message int,message length,wrong %,wrong some,usage frequency,correct frequency
0,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 0, 2]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,70,70
1,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 1, 2]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,70,70
2,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 1, 2]",2,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,70,70
3,1,,1,"[1, 0, 1]","[1, 0, 1]","[0, 2]",2,"[2, -1, 2]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,70,70
4,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 0, 2]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,0,0,70,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",1,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17,8
1527,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",0,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17,8
1528,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",1,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17,8
1529,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",0,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17,8


# Meaning calculation attempt

In [57]:
def many_hot(input,data):
    output = torch.zeros([sum(data.properties_dim)])
    start = 0

    for elem, dim in zip(input, data.properties_dim):
        output[start + elem] = 1
        start += dim

    return output

In [58]:
def get_sender_input(data, concept_idx, context_condition, encoding_func):
        sender_concept, sender_context = data.get_sample(concept_idx, context_condition)

        # subset such that only target objects are presented to sender and receiver
        sender_targets = sender_concept[0]
        sender_input = [obj for obj in sender_targets]

        # append context objects
        # get context of relevant context condition
        for distractor_objects, context_cond in sender_context:
            if context_cond == context_condition:
                # add distractor objects for the sender
                for obj in distractor_objects:
                    sender_input.append(obj)

        # ENCODE and return as TENSOR
        sender_input = torch.stack([many_hot(elem,data) for elem in sender_input])
        return sender_input

In [59]:
def get_receiver_input(data, concept_idx, context_condition, encoding_func):
        receiver_concept, receiver_context = data.get_sample(concept_idx, context_condition)

        # subset such that only target objects are presented to sender and receiver
        receiver_targets = receiver_concept[0]
        receiver_input = [obj for obj in receiver_targets]

        for distractor_objects, context_cond in receiver_context:
            if context_cond == context_condition:
                # add distractor objects for the receiver
                for obj in distractor_objects:
                    receiver_input.append(obj)
        
        random.shuffle(receiver_input)
        receiver_label = [idx for idx, obj in enumerate(receiver_input) if obj in receiver_targets]
        receiver_label = torch.Tensor(receiver_label).to(torch.int64)
        receiver_label = F.one_hot(receiver_label, num_classes=data.game_size * 2).sum(dim=0).float()

        # ENCODE and return as TENSOR
        receiver_input = torch.stack([many_hot(elem,data) for elem in receiver_input])
        return receiver_label, receiver_input

In [60]:
# calculate meanings (concept x message for fine context)
runs = 7
data = DataSet(list(itertools.repeat(values, attributes)),
    game_size=10,
    scaling_factor=10,
    zero_shot=True,
    zero_shot_test=False,
    sample_context=False)

concept_fixed_set = data.get_all_concepts()
set_concepts = [list( map(int, list( [a if fa == 1. else -1 for a,fa in zip(c[0],f) ] ))) for c,f in concept_fixed_set]
fint_set = [list(map(int,list(f))) for c,f in concept_fixed_set]
sender_input = torch.cat([get_sender_input(data, i, sum(f)-1, data._many_hot_encoding).unsqueeze(0) for i,f in zip(range(len(concept_fixed_set)),fint_set)],dim=0)
messages_original = [sender(sender_input) for r in range(runs)]
messages_tensor = torch.cat([messages_without_after_eos(msg,0.0).argmax(dim=-1).unsqueeze(1) for msg in messages_original],dim=1)

Creating train_ds, val_ds and test_ds...


100%|██████████| 124/124 [00:00<00:00, 123244.95it/s]

Length of train and validation datasets: 0 / 0
Length of test dataset: 0


In [61]:
# get whether the listener can understand the messages
receiver_input_list = []
labels_list = []
for i,f in zip(range(len(concept_fixed_set)),fint_set):
    labels, receiver_input = get_receiver_input(data, i, sum(f)-1, data._many_hot_encoding)
    receiver_input_list.append(receiver_input.unsqueeze(0))
    labels_list.append(labels.unsqueeze(0).unsqueeze(0).unsqueeze(0))

receiver_input = torch.cat(receiver_input_list,dim=0)
labels = torch.cat(labels_list,dim=0).repeat(1,runs,attributes + 1, 1)

eos_mask_total = 0.0 != messages_tensor # remember this is fore several runs
# only values before eos, to switch
eos_excluded_cumulative = torch.cat([ eos_mask_total[:,:,:i+1].sum(dim=2).unsqueeze(-1) == i+1 for i in range(attributes +1) ],dim=-1)
# until eos, evaluating changes
eos_included_cumulative = torch.cat([torch.ones_like(eos_excluded_cumulative[:,:,-1]).unsqueeze(-1).bool(), eos_excluded_cumulative[:,:,:-1]],dim=-1)
eos_exact = (eos_included_cumulative != eos_mask_total) # only symbol first eos

predictions = torch.cat([(listener(msg,receiver_input) > 0).float().unsqueeze(1) for msg in messages_original],dim=1)
# remove after eos prediction
predictions = torch.where(eos_exact.unsqueeze(-1),predictions,labels)
score = (predictions != labels)

In [62]:
def aggregate_messages(messages):
    message_counts = messages.value_counts().to_dict()
    unique_messages = list(message_counts.keys())
    frequencies = sum(list(message_counts.values()))
    return unique_messages, frequencies

In [63]:
# make one DataFrame containing all the concepts and messages used for them in fine context

# make data hashable, and calc averages etc. 
messages_set = [list(set([f"{ *mes, }" for mes in msg_5])) for msg_5 in [msg.tolist() for msg in messages_tensor]]
messages_str = [[f"{ *mes, }" for mes in msg_5] for msg_5 in [msg.tolist() for msg in messages_tensor]]
concepts_str = [f"{ *con, }" for con in set_concepts]
wrong_in_total = score.sum(dim=(2,3)).bool().sum(dim=-1)/runs
wrong_per_message = score.sum(dim=(2,3))/20

# create DataFrame to calc average wrongness per message etc
in_list = []
for i in range(len(messages_str)):
    for j,m in enumerate(messages_str[i]):
        in_list.append((m,concepts_str[i],wrong_per_message[i][j].item()))
meaning_message_df = pd.DataFrame(in_list,columns=["message","meaning","wrong %"])
meaning_message_df = meaning_message_df.groupby(['meaning','message'])['wrong %'].apply(lambda l: l.mean()).reset_index().round(2)
with_messages = meaning_message_df.groupby(['meaning'])['message'].apply(lambda m:list(m.value_counts().to_dict().keys())).reset_index()
with_wrong = meaning_message_df.groupby(['meaning'])['wrong %'].apply(lambda m:list(m.value_counts().to_dict().keys())).reset_index()
wrong_per_message_df = with_messages.merge(with_wrong,how='right',on="meaning")

# create DataFrame with all data
in_list = []
for i in range(len(messages_set)):
    in_list.append((concepts_str[i],fint_set[i],wrong_in_total[i].item()))
meaning_df = pd.DataFrame(in_list,columns=["meaning","fixed meaning int","not all correct percent total"]).round(1)
#meaning_df.round(1).drop(meaning_df[meaning_df["not all correct percent total"] == 0.0].index)
meaning_df = meaning_df.merge(wrong_per_message_df,how='right',on='meaning')
meaning_df

,meaning,fixed meaning int,not all correct percent total,message,wrong %
0,"(-1, -1, 0)","[0, 0, 1]",0.0,"[(11, 0, 0, 0)]",[0.0]
1,"(-1, -1, 1)","[0, 0, 1]",1.0,"[(1, 12, 0, 0)]",[0.47]
2,"(-1, -1, 2)","[0, 0, 1]",1.0,"[(0, 0, 0, 0)]",[0.4]
3,"(-1, -1, 3)","[0, 0, 1]",0.0,"[(9, 2, 0, 0)]",[0.0]
4,"(-1, 0, -1)","[0, 1, 0]",1.0,"[(1, 0, 0, 0), (1, 1, 0, 0), (1, 9, 0, 0)]","[0.5, 0.49, 0.45]"
...,...,...,...,...,...
119,"(3, 3, -1)","[1, 1, 0]",0.1,"[(5, 1, 0, 0)]",[0.01]
120,"(3, 3, 0)","[1, 1, 1]",0.0,"[(10, 1, 0, 0)]",[0.0]
121,"(3, 3, 1)","[1, 1, 1]",0.0,"[(5, 11, 0, 0)]",[0.0]
122,"(3, 3, 2)","[1, 1, 1]",1.0,"[(5, 0, 0, 0)]",[0.21]


In [64]:
# create a similar dataframe but sorted by message
with_wrong = meaning_message_df.groupby(['message'])['wrong %'].apply(lambda m:list(m.value_counts().to_dict().keys())).reset_index()
with_meaning = meaning_message_df.groupby(['message'])['meaning'].apply(aggregate_messages).reset_index()
with_meaning[['unique meanings', 'frequency used by meaning']] = pd.DataFrame(with_meaning['meaning'].tolist(), index=with_meaning.index)
message_df = with_meaning.merge(with_wrong,how='right',on="message").drop("meaning",axis=1)
message_df # TODO true meaning with wrong == 0

,message,unique meanings,frequency used by meaning,wrong %
0,"(0, 0, 0, 0)","[(-1, -1, 2), (-1, 1, 2), (-1, 1, 3), (0, 1, 2...",9,"[0.4, 0.08, 0.04, 0.0, 0.11, 0.13, 0.05, 0.06]"
1,"(1, 0, 0, 0)","[(-1, 0, -1), (-1, 0, 2), (-1, 3, 2), (0, 0, 2...",8,"[0.0, 0.05, 0.5, 0.1, 0.18]"
2,"(1, 1, 0, 0)","[(-1, 0, -1), (-1, 3, -1), (1, 3, 0), (2, 3, -...",5,"[0.0, 0.49, 0.21, 0.15]"
3,"(1, 10, 0, 0)","[(-1, 0, 1), (3, 0, 1)]",2,"[0.0, 0.2]"
4,"(1, 11, 0, 0)","[(-1, 3, 1), (0, 3, 0), (0, 3, 1), (1, 3, 0), ...",6,"[0.0, 0.08, 0.07, 0.15, 0.6]"
...,...,...,...,...
86,"(9, 1, 0, 0)","[(-1, 0, 2), (-1, 3, 3)]",2,"[0.0, 0.53]"
87,"(9, 2, 0, 0)","[(-1, -1, 3)]",1,[0.0]
88,"(9, 3, 0, 0)","[(1, 0, 3)]",1,[0.0]
89,"(9, 3, 3, 0)","[(1, 0, 3)]",1,[0.0]


In [ ]:
# find true meaning


# Data Analysis

In [73]:
message_length.float().mean() # weighted by occurence

tensor(1.4386)

In [74]:
# How long are the messages on average used for a certain fix attribute number
scores = message_length_per_hierarchy_level(interaction, attributes)
scores # These values are correct!

message length tensor([1, 1, 2,  ..., 1, 2, 1])
hierarchical [1.597, 1.447, 1.425]


[1.597, 1.447, 1.425]

In [75]:
# How long are the messages on average used in the different context conditions
grouped_by_context = df.groupby(["context condition"])['message length'].apply(lambda l: l.mean()).reset_index()
grouped_by_context_wrong = df.groupby(["context condition"])['wrong %'].apply(lambda l: l.mean()).reset_index()
grouped_by_context_wrong2 = df.groupby(["context condition"])['wrong some'].apply(lambda l: l.mean()).reset_index()
grouped_by_context = grouped_by_context.merge(grouped_by_context_wrong,how='right',on='context condition')
grouped_by_context = grouped_by_context.merge(grouped_by_context_wrong2,how='right',on='context condition')
grouped_by_context

,context condition,message length,wrong %,wrong some
0,0,1.313351,0.483651,0.025886
1,1,1.479769,1.763006,0.148844
2,2,1.612022,4.672131,0.379781


In [76]:
# how long are the messages on average for different numbers of fixed attributes
grouped_by_nr_fixed_attributes = df.groupby(["nr attributes","context condition"])["message length"].apply(lambda l: l.mean()).reset_index()
grouped_by_nr_fixed_attributes_wrong = df.groupby(["nr attributes","context condition"])["wrong %"].apply(lambda l: l.mean()).reset_index()
grouped_by_nr_fixed_attributes_wrong2 = df.groupby(["nr attributes","context condition"])["wrong some"].apply(lambda l: l.mean()).reset_index()
grouped_by_nr_fixed_attributes = grouped_by_nr_fixed_attributes.merge(grouped_by_nr_fixed_attributes_wrong,how='right',on=['nr attributes', 'context condition'])
grouped_by_nr_fixed_attributes = grouped_by_nr_fixed_attributes.merge(grouped_by_nr_fixed_attributes_wrong2,how='right',on=['nr attributes', 'context condition'])
grouped_by_nr_fixed_attributes

,nr attributes,context condition,message length,wrong %,wrong some
0,1,0,1.596774,4.758065,0.258065
1,2,0,1.325658,0.197368,0.009868
2,2,1,1.565079,3.174603,0.241270
3,3,0,1.255435,0.000000,0.000000
4,3,1,1.408488,0.583554,0.071618
5,3,2,1.612022,4.672131,0.379781


In [77]:
# how often are messages used with different lengths in total
#grouped_by_length = pd.DataFrame([(l,f) for l,f in zip(ul,f)],columns=["length","frequency"])
grouped_by_length = df.groupby(["message length"])['usage frequency'].apply(lambda f: (f.mean(),f.std(ddof=False))).reset_index()
grouped_by_length_wrong = df.groupby(["message length"])['wrong %'].apply(lambda f: f.mean()).reset_index()
grouped_by_length_wrong2 = df.groupby(["message length"])['wrong some'].apply(lambda f: f.mean()).reset_index()
grouped_by_length[['mean usage frequency', 'std usage frequency']] = pd.DataFrame(grouped_by_length['usage frequency'].tolist(), index=grouped_by_length.index)
grouped_by_length = grouped_by_length.merge(grouped_by_length_wrong,how='right',on='message length')
grouped_by_length = grouped_by_length.merge(grouped_by_length_wrong2,how='right',on='message length')
grouped_by_length = grouped_by_length.drop('usage frequency',axis=1)
grouped_by_length

,message length,mean usage frequency,std usage frequency,wrong %,wrong some
0,0,70.000000,0.000000,12.357143,0.828571
1,1,69.075949,27.636157,1.173763,0.109321
2,2,17.204706,10.020942,1.647059,0.127059
3,3,1.000000,0.000000,0.000000,0.000000


In [78]:
df_attributes = df[df['context condition'] + 1 == df['nr attributes']]
# Idee: remove all messages with wrong some, so that wrongly used messages or estimations are excluded. Because they mean something different. 
#df_attributes = df_attributes[df_attributes['wrong some'] == 0] # Done at the top, if not there you can use it here. 
df_attributes

,fixed 1,fixed 2,fixed 3,fixed,fixed int,which fixed indices,nr attributes,concept,context condition,message,message int,message length,wrong %,wrong some,usage frequency
0,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 2]",2,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,10,1,70
1,1,1,1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 1, 2]",2,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,30,1,70
3,,1,1,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 1, 2]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,20,1,70
4,1,,1,"[1, 0, 1]","[1, 0, 1]","[0, 2]",2,"[2, -1, 2]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,30,1,70
5,,1,1,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 1, 3]",1,"(0, 0, 0, 0)","[0, 0, 0, 0]",0,10,1,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",1,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17
1783,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",1,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17
1785,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",1,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17
1787,1,1,,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[1, 2, -1]",1,"(15, 15, 0, 0)","[15, 15, 0, 0]",2,0,0,17


In [79]:
df_attributes_grouped = df_attributes.groupby(['fixed 1','fixed 2','fixed 3'])['message length'].apply(lambda l: l.mean()).reset_index()
df_attributes_frequency = df_attributes.groupby(['fixed 1','fixed 2','fixed 3'])['usage frequency'].apply(lambda l: l.sum()).reset_index()
df_attributes_wrong = df_attributes.groupby(['fixed 1','fixed 2','fixed 3'])['wrong %'].apply(lambda l: l.mean()).reset_index()
df_attributes_wrong2 = df_attributes.groupby(['fixed 1','fixed 2','fixed 3'])['wrong some'].apply(lambda l: l.mean()).reset_index()
df_attributes = df_attributes_grouped.merge(df_attributes_frequency,how='right',on=['fixed 1','fixed 2', 'fixed 3'])
df_attributes = df_attributes.merge(df_attributes_wrong,how='right',on=['fixed 1','fixed 2', 'fixed 3'])
df_attributes = df_attributes.merge(df_attributes_wrong2,how='right',on=['fixed 1','fixed 2', 'fixed 3'])
df_attributes

,fixed 1,fixed 2,fixed 3,message length,usage frequency,wrong %,wrong some
0,1,1,1,1.612022,10164,4.672131,0.379781
1,1,1,,1.858586,2436,1.464646,0.111111
2,1,,1,1.477477,3578,1.666667,0.090090
3,1,,,1.461538,632,0.000000,0.000000
4,,1,1,1.380952,3897,6.380952,0.523810
5,,1,,1.741935,1200,9.516129,0.516129
6,,,1,1.444444,1200,0.000000,0.000000
